# 1. Data gathering

In [108]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

from typing import Dict, Tuple

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV

from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, precision_recall_curve


In [34]:
df = pd.read_csv('data/telco_churn.csv')

In [53]:
df.TotalCharges = pd.to_numeric(df.TotalCharges, errors='coerce')

In [54]:
df.loc[df['TotalCharges'].isna(), 'TotalCharges']= (df.loc[df['TotalCharges'].isna(), 'MonthlyCharges'] * df.loc[df['TotalCharges'].isna(), 'tenure'])

In [55]:
X = df.drop(columns=['Churn', 'customerID'])

In [56]:
X.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65


In [57]:
y = df.Churn.map({'Yes':1, 'No':0})
y.head()

0    0
1    0
2    1
3    0
4    1
Name: Churn, dtype: int64

In [ ]:
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

preprocessor = ColumnTransformer(
    [
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), cat_features),
        ('num', StandardScaler(), num_features)
    ]
)

In [59]:
X_enc = preprocessor.fit_transform(X)

In [60]:
X_enc.shape

(7043, 30)

In [74]:
# seperate dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train.shape, X_test.shape

((5634, 19), (1409, 19))

### Create an evaluate function to give all metrics give all metric after model Training

In [75]:
models = {
    "LogisticRegression": LogisticRegression(max_iter=2000),
    "KNN": KNeighborsClassifier(),
    "SVM (RBF)": SVC(probability=True),
    "DecisionTree": DecisionTreeClassifier(),
    "RandomForest": RandomForestClassifier(n_jobs=-1),
    "GradientBoosting": GradientBoostingClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "GaussianNB": GaussianNB(),
    "MLPClassifier": MLPClassifier(max_iter=1000),
    "XGBoost": XGBClassifier(
        use_label_encoder=False,
        eval_metric="logloss",
        random_state=42
    ),
}

In [76]:
def evaluate_model(model, X_test, y_test, verbose=True, pos_label=1):
    """
    Evaluate a trained classifier on test data.
    Returns a dict with accuracy, precision, recall, f1, auc, and cm.
    Works with pipelines or plain estimators.
    """
    y_pred = model.predict(X_test)

    # Probabilities or decision scores for AUC
    y_score = None
    if hasattr(model, "predict_proba"):
        y_score = model.predict_proba(X_test)[:, 1]
    elif hasattr(model, "decision_function"):
        # scale to [0,1] for ROC-AUC stability
        s = model.decision_function(X_test)
        s = (s - s.min()) / (s.max() - s.min() + 1e-12)
        y_score = s

    acc  = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0, pos_label=pos_label)
    rec  = recall_score(y_test, y_pred, zero_division=0, pos_label=pos_label)
    f1   = f1_score(y_test, y_pred, zero_division=0, pos_label=pos_label)
    auc  = roc_auc_score(y_test, y_score) if y_score is not None else np.nan
    cm   = confusion_matrix(y_test, y_pred)

    if verbose:
        print("=== Evaluation Report ===")
        print("Classification Report:")
        print(classification_report(y_test, y_pred, zero_division=0))
        print(f"Accuracy   : {acc:.4f}")
        print(f"Precision  : {prec:.4f}")
        print(f"Recall     : {rec:.4f}")
        print(f"F1 Score   : {f1:.4f}")
        if not np.isnan(auc):
            print(f"AUC-ROC    : {auc:.4f}")
        print("Confusion Matrix:\n", cm)

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1_score": f1,
        "auc_roc": auc,
        "confusion_matrix": cm
    }

In [77]:
rows = []
trained = {}

for name, clf in models.items():
    pipe = Pipeline([
        ("preprocessor", preprocessor),
        ("clf", clf),
    ])
    pipe.fit(X_train, y_train)
    trained[name] = pipe

    # evaluate_model expects (model, X_test, y_test)
    metrics = evaluate_model(pipe, X_test, y_test, verbose=False)
    metrics["model"] = name
    rows.append(metrics)

    # quick glance
    print(f"\n=== {name} ===")
    print(pd.Series(metrics))




=== LogisticRegression ===
accuracy                            0.806955
precision                           0.660377
recall                              0.561497
f1_score                            0.606936
auc_roc                             0.842171
confusion_matrix    [[927, 108], [164, 210]]
model                     LogisticRegression
dtype: object

=== KNN ===
accuracy                            0.766501
precision                           0.559682
recall                              0.564171
f1_score                            0.561917
auc_roc                             0.792473
confusion_matrix    [[869, 166], [163, 211]]
model                                    KNN
dtype: object

=== SVM (RBF) ===
accuracy                           0.792761
precision                          0.649635
recall                             0.475936
f1_score                           0.549383
auc_roc                            0.792777
confusion_matrix    [[939, 96], [196, 178]]
model             

/Users/tanvir15/Documents/12 month ai challange/machine learning mvps/chun prediction 2/ml_venv/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [04:04:06] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [78]:
# summary table
results_df = (
    pd.DataFrame(rows)
      .set_index("model")
      .sort_values(by=["auc_roc", "f1_score", "accuracy"], ascending=False)
)
results_df

,accuracy,precision,recall,f1_score,auc_roc,confusion_matrix
model,,,,,,
LogisticRegression,0.806955,0.660377,0.561497,0.606936,0.842171,"[[927, 108], [164, 210]]"
GradientBoosting,0.797729,0.653979,0.505348,0.570136,0.841512,"[[935, 100], [185, 189]]"
AdaBoost,0.804826,0.662295,0.540107,0.594993,0.840690,"[[932, 103], [172, 202]]"
RandomForest,0.788502,0.631034,0.489305,0.551205,0.824335,"[[928, 107], [191, 183]]"
XGBoost,0.772179,0.583072,0.497326,0.536797,0.822384,"[[902, 133], [188, 186]]"
GaussianNB,0.655784,0.426877,0.866310,0.571933,0.809236,"[[600, 435], [50, 324]]"
MLPClassifier,0.772889,0.586538,0.489305,0.533528,0.808836,"[[906, 129], [191, 183]]"
SVM (RBF),0.792761,0.649635,0.475936,0.549383,0.792777,"[[939, 96], [196, 178]]"
KNN,0.766501,0.559682,0.564171,0.561917,0.792473,"[[869, 166], [163, 211]]"


# Bining and DisCretization

In [90]:
def test(X):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)
    rows = []
    trained = {}

    for name, clf in models.items():
        pipe = Pipeline([
            ("preprocessor", preprocessor),
            ("clf", clf),
        ])
        pipe.fit(X_train, y_train)
        trained[name] = pipe

        # evaluate_model expects (model, X_test, y_test)
        metrics = evaluate_model(pipe, X_test, y_test, verbose=False)
        metrics["model"] = name
        rows.append(metrics)

        # quick glance
        # print(f"\n=== {name} ===")
        # print(pd.Series(metrics))
    results_df = (
    pd.DataFrame(rows)
    .set_index("model")
    .sort_values(by=["auc_roc", "f1_score", "accuracy"], ascending=False)
    )
    return results_df

In [83]:
X.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692,2279.734304
std,0.368612,24.559481,30.090047,2266.794470
min,0.000000,0.000000,18.250000,0.000000
25%,0.000000,9.000000,35.500000,398.550000
50%,0.000000,29.000000,70.350000,1394.550000
75%,0.000000,55.000000,89.850000,3786.600000
max,1.000000,72.000000,118.750000,8684.800000


In [ ]:
X['tenure_bin'] = pd.cut(X['tenure'], bins = [-1, 15, 29, 43, 57, 72], labels=[1,2,3,4,5])
X['MonthlyCharges_bin'] = pd.cut(X['MonthlyCharges'], bins=[10, 38.5, 58.45, 78.55, 98.65, 120], labels=[1,2,3,4,5])
X['TotalCharges_bin'] = pd.cut(X['TotalCharges'], bins=[-1, 1736.96, 3473.92, 5210.88, 6947.84, np.inf], labels=[1,2,3,4,5])

In [91]:
test(X)

/Users/tanvir15/Documents/12 month ai challange/machine learning mvps/chun prediction 2/ml_venv/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [04:25:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,accuracy,precision,recall,f1_score,auc_roc,confusion_matrix
model,,,,,,
AdaBoost,0.811214,0.690141,0.524064,0.595745,0.857979,"[[947, 88], [178, 196]]"
GradientBoosting,0.808375,0.684397,0.516043,0.588415,0.851935,"[[946, 89], [181, 193]]"
LogisticRegression,0.801987,0.649842,0.550802,0.596237,0.851629,"[[924, 111], [168, 206]]"
RandomForest,0.792051,0.652830,0.462567,0.541471,0.829742,"[[943, 92], [201, 173]]"
XGBoost,0.784954,0.618729,0.494652,0.549777,0.825038,"[[921, 114], [189, 185]]"
GaussianNB,0.687012,0.454297,0.890374,0.601626,0.824475,"[[635, 400], [41, 333]]"
MLPClassifier,0.776437,0.597360,0.483957,0.534712,0.809176,"[[913, 122], [193, 181]]"
SVM (RBF),0.803407,0.673835,0.502674,0.575804,0.802936,"[[944, 91], [186, 188]]"
KNN,0.757275,0.545455,0.513369,0.528926,0.777368,"[[875, 160], [182, 192]]"


## Polynomial Featrues

In [94]:
from scipy.stats import skew
print(skew(X['TotalCharges']))
print(skew(X['tenure']))
print(skew(X['MonthlyCharges']))

0.9630294954586066
0.2394887299846216
-0.2204774644391769


In [95]:
X['log_total_charges'] = np.sqrt(df['TotalCharges']) 
X['log_tenure'] = np.log1p(df['tenure'])
X['log_Monthly_charge'] = np.log1p(df['MonthlyCharges'])

In [97]:
test(X)

/Users/tanvir15/Documents/12 month ai challange/machine learning mvps/chun prediction 2/ml_venv/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [04:29:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,accuracy,precision,recall,f1_score,auc_roc,confusion_matrix
model,,,,,,
AdaBoost,0.811214,0.690141,0.524064,0.595745,0.857979,"[[947, 88], [178, 196]]"
GradientBoosting,0.808375,0.684397,0.516043,0.588415,0.851695,"[[946, 89], [181, 193]]"
LogisticRegression,0.801987,0.649842,0.550802,0.596237,0.851629,"[[924, 111], [168, 206]]"
RandomForest,0.792761,0.650735,0.473262,0.547988,0.827882,"[[940, 95], [197, 177]]"
XGBoost,0.784954,0.618729,0.494652,0.549777,0.825038,"[[921, 114], [189, 185]]"
GaussianNB,0.687012,0.454297,0.890374,0.601626,0.824475,"[[635, 400], [41, 333]]"
MLPClassifier,0.769340,0.572271,0.518717,0.544180,0.807949,"[[890, 145], [180, 194]]"
SVM (RBF),0.803407,0.673835,0.502674,0.575804,0.802933,"[[944, 91], [186, 188]]"
KNN,0.757275,0.545455,0.513369,0.528926,0.777368,"[[875, 160], [182, 192]]"


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)


base_pipe = Pipeline(
    [
        ("preprocessor", preprocessor),
        ('clf', AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1, random_state=0, class_weight='balanced'), random_state=0))
    ]
)

In [119]:
param_grid_ada = {
    "clf__n_estimators": [50, 100, 200, 400],
    "clf__learning_rate": [0.01, 0.05, 0.1, 0.3, 1.0],
    "clf__estimator__max_depth": [1, 2, 3],           
    "clf__estimator__min_samples_leaf": [1, 5, 10],
}

In [120]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [121]:
gs_ada = GridSearchCV(
    estimator=base_pipe,
    param_grid=param_grid_ada,
    scoring={
        "roc_auc": "roc_auc",
        "f1": "f1",
        "recall": "recall",
        "precision": "precision",
        "accuracy": "accuracy",
    },
    refit="roc_auc",     # the model re-fitted on the whole train with best roc_auc
    cv=cv,
    n_jobs=-1,
    verbose=1
)

In [129]:
gs_ada.fit(X_train, y_train)

Fitting 5 folds for each of 180 candidates, totalling 900 fits


/Users/tanvir15/Documents/12 month ai challange/machine learning mvps/chun prediction 2/ml_venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/tanvir15/Documents/12 month ai challange/machine learning mvps/chun prediction 2/ml_venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/tanvir15/Documents/12 month ai challange/machine learning mvps/chun prediction 2/ml_venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is i

,estimator,Pipeline(step...om_state=0))])
,param_grid,"{'clf__estimator__max_depth': [1, 2, ...], 'clf__estimator__min_samples_leaf': [1, 5, ...], 'clf__learning_rate': [0.01, 0.05, ...], 'clf__n_estimators': [50, 100, ...]}"
,scoring,"{'accuracy': 'accuracy', 'f1': 'f1', 'precision': 'precision', 'recall': 'recall', ...}"
,n_jobs,-1
,refit,'roc_auc'
,cv,StratifiedKFo... shuffle=True)
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('cat', ...), ('num', ...)]"


In [130]:
print("Best AdaBoost params:", gs_ada.best_params_)
print("Best CV ROC-AUC     :", gs_ada.best_score_)

Best AdaBoost params: {'clf__estimator__max_depth': 3, 'clf__estimator__min_samples_leaf': 5, 'clf__learning_rate': 0.1, 'clf__n_estimators': 200}
Best CV ROC-AUC     : 0.8496750310844801


In [131]:
# Evaluate on holdout
best_ada = gs_ada.best_estimator_
y_pred = best_ada.predict(X_test)
y_proba = best_ada.predict_proba(X_test)[:, 1]

In [132]:
print("\n=== Holdout Evaluation (AdaBoost) ===")
print("Accuracy :", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall   :", recall_score(y_test, y_pred))
print("F1       :", f1_score(y_test, y_pred))
print("ROC-AUC  :", roc_auc_score(y_test, y_proba))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


=== Holdout Evaluation (AdaBoost) ===
Accuracy : 0.8112136266855926
Precision: 0.6888111888111889
Recall   : 0.5267379679144385
F1       : 0.5969696969696969
ROC-AUC  : 0.8540429874189466

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.91      0.88      1035
           1       0.69      0.53      0.60       374

    accuracy                           0.81      1409
   macro avg       0.77      0.72      0.74      1409
weighted avg       0.80      0.81      0.80      1409

Confusion Matrix:
 [[946  89]
 [177 197]]


In [127]:
#see top CV rows
cv_results = pd.DataFrame(gs_ada.cv_results_).sort_values(by="mean_test_roc_auc", ascending=False)
cv_results.head(10)[[
    "mean_test_roc_auc","mean_test_f1","mean_test_recall","mean_test_precision","mean_test_accuracy",
    "param_clf__n_estimators","param_clf__learning_rate","param_clf__estimator__max_depth","param_clf__estimator__min_samples_leaf"
]]

,mean_test_roc_auc,mean_test_f1,mean_test_recall,mean_test_precision,mean_test_accuracy,param_clf__n_estimators,param_clf__learning_rate,param_clf__estimator__max_depth,param_clf__estimator__min_samples_leaf
150,0.849675,0.581081,0.519064,0.661987,0.802095,200,0.10,3,5
131,0.849498,0.586884,0.529097,0.661138,0.802982,400,0.10,3,1
151,0.849452,0.585132,0.527090,0.659474,0.802272,400,0.10,3,5
130,0.849434,0.582214,0.520401,0.662573,0.802450,200,0.10,3,1
171,0.849346,0.583530,0.525084,0.658475,0.801562,400,0.10,3,10
149,0.849317,0.577156,0.509699,0.667027,0.802272,100,0.10,3,5
129,0.849167,0.580110,0.515050,0.666067,0.802627,100,0.10,3,1
169,0.849089,0.581624,0.517057,0.666556,0.803160,100,0.10,3,10
170,0.849042,0.583609,0.521070,0.665566,0.803515,200,0.10,3,10
147,0.848758,0.582644,0.519732,0.665153,0.803159,400,0.05,3,5
